In [1]:
import findspark 
findspark.init()
findspark.find()

'C:\\spark-3.4.0-bin-hadoop3'

In [2]:
import pyspark 
from pyspark import SparkContext
sc = SparkContext("local","Matrix Vector Multiplication")

In [27]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, posexplode, collect_list, first
spark = SparkSession.builder.getOrCreate()

In [99]:
matrix1 = sc.textFile(r"C:\Users\Gurudeep\Desktop\spark-warehouse\Matrix.txt")
matrix2 = sc.textFile(r"C:\Users\Gurudeep\Desktop\spark-warehouse\Vector.txt")

In [100]:
matrix1.take(6)

['1 2 2.5', '1 3 1', '2 1 4', '2 4 -2', '3 3 -1.5', '4 1 8']

In [101]:
matrix2.take(3)

['1', '0', '2']

In [102]:
matrix1 = matrix1.map(lambda line: line.split()).map(lambda cols: [float(cols[0]), float(cols[1]), float(cols[2])])
matrix2 = matrix2.map(int)

In [103]:
matrix1.take(6)

[[1.0, 2.0, 2.5],
 [1.0, 3.0, 1.0],
 [2.0, 1.0, 4.0],
 [2.0, 4.0, -2.0],
 [3.0, 3.0, -1.5],
 [4.0, 1.0, 8.0]]

In [104]:
matrix2.take(3)

[1, 0, 2]

In [105]:
# Define the multiplication factors
result_rdd = matrix1.zip(matrix2).map(lambda x: [elem * factor for elem, factor in zip(x[0], x[1])])

In [106]:
matrix1_list = matrix1.collect()
transposed_matrix = list(zip(*matrix1_list))
# Define the second RDD
#rdd2 = spark.sparkContext.parallelize([2, 4, 6])

# Create an RDD from the matrix
rdd1 = spark.sparkContext.parallelize(transposed_matrix)

# Multiply the matrix (rdd1) with the second RDD (rdd2) element-wise
result_rdd = rdd1.zip(matrix2).map(lambda x: [elem * x[1] for elem in x[0]])

In [107]:
transposed_matrix

[(1.0, 1.0, 2.0, 2.0, 3.0, 4.0),
 (2.0, 3.0, 1.0, 4.0, 3.0, 1.0),
 (2.5, 1.0, 4.0, -2.0, -1.5, 8.0)]

In [108]:
result_rdd.take(3)

[[1.0, 1.0, 2.0, 2.0, 3.0, 4.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [5.0, 2.0, 8.0, -4.0, -3.0, 16.0]]

In [110]:
column_matrix_rdd = spark.sparkContext.parallelize([
    result_rdd.map(lambda sublist: sublist[0]).reduce(lambda a, b: a + b),
    result_rdd.map(lambda sublist: sublist[1]).reduce(lambda a, b: a + b),
    result_rdd.map(lambda sublist: sublist[2]).reduce(lambda a, b: a + b),
    result_rdd.map(lambda sublist: sublist[3]).reduce(lambda a, b: a + b),
    result_rdd.map(lambda sublist: sublist[4]).reduce(lambda a, b: a + b),
    result_rdd.map(lambda sublist: sublist[5]).reduce(lambda a, b: a + b)
])

In [111]:
column_matrix_rdd.take(6)

[6.0, 3.0, 10.0, -2.0, 0.0, 20.0]